## Train the model

In [1]:
from utils import dataloader
import utils.model
from datetime import datetime
from os.path import join
from importlib import reload

method_name = "26_normal_noise"
filter_timestamp = None#int(datetime(2021, 2, 19, 0).timestamp())
use_user_index = join("indices","user_index_w_type.parquet")#"train_user_index.parquet"
csv_data_location = join("data","downloaded_data")
model_save_location = join("saved_models","xgb_models_"+method_name)
result_logfile = join("logs","3batches_results.txt")

In [2]:
dl = dataloader.RecSys2021TSVDataLoader(
    csv_data_location, 
    use_user_index, 
    mode="train", 
    filter_timestamp=filter_timestamp, 
    verbose=2, 
    remove_day_counts=True, 
    keep_user_percent=0.98, 
    batch_size = 1500000,
    load_n_batches = 1,
    TE_smoothing = {"reply":50, "like":50, "retweet":50, "retweet_comment":50},
    remove_user_counts=True,
    add_normal_TE_noise_std= 0.1
    )

Loading User Index
Randomly keeping only 98.0% of the users.
Removing day counts
Extracting combined user counts
Created Dataloader in 62.04 seconds!


In [3]:
recsysxgb = utils.model.RecSysXGB1(model_dir = model_save_location)
xgb_params = {'objective': 'binary:logistic', 'eval_metric':'logloss', 'max_depth':8, "max_delta_step":5}

In [4]:
%%time
recsysxgb.fit(
    train_loader= dl,
    xgb_parameters = xgb_params,
    boost_rounds_per_iteration = 25,
    verbose = 1,
    n_epochs = 1
)

Loaded Batch Nr. 1 in 11.54
Timestamp Filtered Batch Nr. 1 in 0.00
Did prepro part 1 of 1 in 0.37
Did prepro part 2 of 1 in 19.89
Did prepro part 3 of 1 in 0.09
Merged Users of 1 in 29.89
Extracted TE of 1 in 18.45
Finished Batch Nr. 1 from file part-00000.tsv in 83.33s!
Finished 1 epochs.
CPU times: user 1h 49min 59s, sys: 1min 8s, total: 1h 51min 7s
Wall time: 19min 2s


## Evaluate the model

In [5]:
recsysxgb = utils.model.RecSysXGB1(model_save_location)
val_data_location = join("data","validation_data")

dl = dataloader.RecSys2021TSVDataLoader(
    data_directory = val_data_location, 
    user_index_location = use_user_index,
    mode="val", 
    filter_timestamp=filter_timestamp, 
    verbose=2, 
    random_file_sampling=False, 
    load_n_batches=3, 
    remove_day_counts=True,
    batch_size = 1000000,
    keep_user_percent=1,
    remove_user_counts=True
)

Loading User Index
Removing day counts
Extracting combined user counts
Created Dataloader in 17.37 seconds!


In [6]:
res = recsysxgb.evaluate_validation_set(dl, store_results_file=result_logfile, validation_run_name=method_name)

Loaded Batch Nr. 1 in 9.60
Timestamp Filtered Batch Nr. 1 in 0.00
Did prepro part 1 of 1 in 0.24
Did prepro part 2 of 1 in 12.95
Did prepro part 3 of 1 in 0.06
Merged Users of 1 in 26.75
Extracted TE of 1 in 14.53
Finished Batch Nr. 1 from file part-00000.csv in 66.29s!
Loaded Batch Nr. 2 in 8.41
Timestamp Filtered Batch Nr. 2 in 0.00
Did prepro part 1 of 2 in 0.23
Did prepro part 2 of 2 in 13.72
Did prepro part 3 of 2 in 0.06
Merged Users of 2 in 26.90
Extracted TE of 2 in 14.60
Finished Batch Nr. 2 from file part-00000.csv in 66.14s!
Loaded Batch Nr. 3 in 9.92
Timestamp Filtered Batch Nr. 3 in 0.00
Did prepro part 1 of 3 in 0.23
Did prepro part 2 of 3 in 12.86
Did prepro part 3 of 3 in 0.06
Merged Users of 3 in 26.11
Extracted TE of 3 in 14.48
Finished Batch Nr. 3 from file part-00000.csv in 65.87s!


In [7]:
for key in res:
    print(f"{key:32}: {res[key]}")

Q1_reply_rce                    : 16.797255603849504
Q1_retweet_rce                  : 17.84944501996282
Q1_retweet_comment_rce          : 7.346209054571551
Q1_like_rce                     : 8.10435535071986
Q1_reply_avg_prec               : 0.15672771595875065
Q1_retweet_avg_prec             : 0.34807501659488777
Q1_retweet_comment_avg_prec     : 0.032727663041728305
Q1_like_avg_prec                : 0.5737051328069082
Q2_reply_rce                    : 17.642558663218622
Q2_retweet_rce                  : 18.53405541099886
Q2_retweet_comment_rce          : 7.16232480681489
Q2_like_rce                     : 6.690560483830488
Q2_reply_avg_prec               : 0.17692840174294674
Q2_retweet_avg_prec             : 0.34011477720675376
Q2_retweet_comment_avg_prec     : 0.03215066514415215
Q2_like_avg_prec                : 0.5545472446376853
Q3_reply_rce                    : 19.548606830474014
Q3_retweet_rce                  : 19.669318098613886
Q3_retweet_comment_rce          : 7.82525003432

In [8]:
for target in recsysxgb.targets__:
    print(f"\n{target}:")
    print(dict(sorted(recsysxgb.clfs_[target].get_score(importance_type='gain').items(), key=lambda item: item[1], reverse=True)))


has_reply:
{'a_follows_b': 816.8727754674998, 'type_encoding': 698.7584911069767, 'TE_reply_b_TopLevel_B': 79.17604270813766, 'TE_reply_b_B': 62.9810340630632, 'TE_reply_a_TopLevel_A': 47.11173821669586, 'photo_count': 44.71449772142858, 'language_encoding': 37.563772370786516, 'TE_reply_a_A': 33.0878790388255, 'a_follower_count': 32.53890980251586, 'TE_like_b_TopLevel_B': 28.264533574417275, 'TE_like_b_B': 26.166588066086533, 'a_following_count': 24.044217551615375, 'a_b_follower_ratio': 23.798618769, 'TE_like_b_A': 20.612591891060603, 'TE_like_a_A': 19.181902940800004, 'TE_retweet_b_B': 18.78931545011905, 'bert_token_len': 18.49117528077612, 'TE_like_b_TopLevel_A': 17.817566853570252, 'TE_like_a_TopLevel_A': 17.661627712989695, 'b_creation_delta': 16.310241203855565, 'TE_reply_b_Quote_B': 15.777532801475607, 'TE_reply_a_Quote_A': 15.586299199583337, 'TE_retweet_b_TopLevel_B': 15.349920673897959, 'a_creation_delta': 14.84359915544445, 'TE_retweet_a_TopLevel_A': 14.097147418023257, 'T

## Try sample test run

In [22]:
out = next(iter(dl))

Loaded Batch Nr. 1 in 20.16
Timestamp Filtered Batch Nr. 1 in 0.41
Did prepro part 1 of 1 in 0.27
Did prepro part 2 of 1 in 10.67
Did prepro part 3 of 1 in 0.05
Merged Users of 1 in 23.01
Extracted TE of 1 in 13.02
Finished Batch Nr. 1 from file part-00073.tsv in 70.12s!


In [26]:
pred = recsysxgb.infer(out[0])

In [37]:
(((pred[3]-0.5)*0.9999999)+0.5).min()

5.9604645e-08

## Testing Custom batch sizes

In [1]:
from utils import dataloader
from utils import dataloader
import utils.model
from datetime import datetime
from os.path import join

filter_timestamp = None#int(datetime(2021, 2, 19, 0).timestamp())
use_user_index = join("indices","user_index.parquet")#"train_user_index.parquet"
csv_data_location = join("data","test_files")
model_save_location = join("saved_models","xgb_models_06_submission")

In [2]:
dl = dataloader.RecSys2021TSVDataLoader(csv_data_location, use_user_index, mode="test", filter_timestamp=filter_timestamp, load_n_batches=-1, batch_size=1000000, verbose=2, random_file_sampling=True)

In [3]:
r = [a[1] for a in dl]

just one
1000000
Loaded Batch Nr. 1 in 6.10
Timestamp Filtered Batch Nr. 1 in 0.00
Did prepro part 1 of 1 in 0.20
Did prepro part 2 of 1 in 11.80
Did prepro part 3 of 1 in 0.05
Merged Users of 1 in 21.76
Extracted TE of 1 in 3.21
Finished Batch Nr. 1 from file part-00003.csv in 45.47s!
just one
1000000
Loaded Batch Nr. 2 in 7.32
Timestamp Filtered Batch Nr. 2 in 0.00
Did prepro part 1 of 2 in 0.20
Did prepro part 2 of 2 in 11.87
Did prepro part 3 of 2 in 0.05
Merged Users of 2 in 22.05
Extracted TE of 2 in 3.20
Finished Batch Nr. 2 from file part-00003.csv in 47.02s!
just one
1000000
Loaded Batch Nr. 3 in 8.79
Timestamp Filtered Batch Nr. 3 in 0.00
Did prepro part 1 of 3 in 0.20
Did prepro part 2 of 3 in 11.85
Did prepro part 3 of 3 in 0.05
Merged Users of 3 in 22.73
Extracted TE of 3 in 3.20
Finished Batch Nr. 3 from file part-00003.csv in 49.16s!
more than one 1000000
1000000
Loaded Batch Nr. 4 in 12.99
Timestamp Filtered Batch Nr. 4 in 0.00
Did prepro part 1 of 4 in 0.20
Did prepro 

In [4]:
for df in r:
    print(len(df))

1000000
1000000
1000000
1000000
1000000
1000000
21788


In [17]:
import utils.features as fe
import utils.constants as co
import pandas as pd
import numpy as np

In [12]:
current_file = pd.read_csv(
                "data/test_files/part-00002.csv",
                sep='\x01',
                header=None,
                names=co.all_features,
                dtype={k: v for k, v in co.dtypes_of_features.items() if k in co.all_features}
            )

In [14]:
current_file["medias"] = current_file["medias"].fillna("")
current_file["hashtags"] = current_file["hashtags"].fillna("")
current_file["links"] = current_file["links"].fillna("")
current_file["domains"] = current_file["domains"].fillna("")
current_file["medias"] = current_file["medias"].fillna("")

In [27]:
import numpy as np
set(np.exp(train_data["n_gifs"]+ train_data["n_photos"] +train_data["n_videos"])-1)

{-0.9502129554748535, 1.0, 2.0, 3.0, 4.000000476837158, 5.0, 7.0}

# Other

In [3]:
from utils import dataloader
import utils.model
from datetime import datetime
from os.path import join
from importlib import reload
import torch

filter_timestamp = int(datetime(2021, 2, 19, 0).timestamp())
use_user_index = join("indices","train_user_index_w_type.parquet")#"train_user_index.parquet"
csv_data_location = join("data","downloaded_data")
model_save_location = join("saved_models","xgb_models_15_better_index")

In [2]:
dl = dataloader.RecSys2021TSVDataLoader(
    csv_data_location, 
    use_user_index, 
    mode="train", 
    filter_timestamp=filter_timestamp, 
    verbose=2, 
    remove_day_counts=True, 
    keep_user_percent=0.5, 
    batch_size = 100000,
    minibatches_size = 64,
    load_n_batches = 20,
    normalize_batch = True
    )

Loading User Index
Randomly keeping only 50.0% of the users.
Removing day counts
Extracting combined user counts
Created Dataloader in 21.99 seconds!


In [3]:
nn = utils.model.RecSysNeural1("saved_models/nn1",166).to("cuda")

In [4]:
nn.fit(dl, 1)

Loaded Batch Nr. 1 in 0.53
Timestamp Filtered Batch Nr. 1 in 0.02
Did prepro part 1 of 1 in 0.02
Did prepro part 2 of 1 in 0.88
Did prepro part 3 of 1 in 0.01
Merged Users of 1 in 8.78
Extracted TE of 1 in 5.36
Finished Batch Nr. 1 from file part-00000.tsv in 15.86s!
Loaded Batch Nr. 2 in 0.61
Timestamp Filtered Batch Nr. 2 in 0.02
Did prepro part 1 of 2 in 0.02
Did prepro part 2 of 2 in 0.88
Did prepro part 3 of 2 in 0.01
Merged Users of 2 in 8.81
Extracted TE of 2 in 5.29
Finished Batch Nr. 2 from file part-00000.tsv in 15.91s!
Loaded Batch Nr. 3 in 1.41
Timestamp Filtered Batch Nr. 3 in 0.02
Did prepro part 1 of 3 in 0.02
Did prepro part 2 of 3 in 0.86
Did prepro part 3 of 3 in 0.00
Merged Users of 3 in 8.80
Extracted TE of 3 in 5.32
Finished Batch Nr. 3 from file part-00000.tsv in 16.70s!
Loaded Batch Nr. 4 in 1.51
Timestamp Filtered Batch Nr. 4 in 0.02
Did prepro part 1 of 4 in 0.02
Did prepro part 2 of 4 in 0.87
Did prepro part 3 of 4 in 0.01
Merged Users of 4 in 8.77
Extracted T

In [5]:
dl = dataloader.RecSys2021TSVDataLoader(
    csv_data_location, 
    use_user_index, 
    mode="val", 
    filter_timestamp=filter_timestamp, 
    verbose=2,
    remove_day_counts=True, 
    keep_user_percent=1, 
    batch_size = 100000,
    minibatches_size = 64,
    load_n_batches = 2,
    normalize_batch = True
)

nn.evaluate_validation_set(dl)

Loading User Index
Removing day counts
Extracting combined user counts
Created Dataloader in 9.23 seconds!
Loaded Batch Nr. 1 in 0.50
Timestamp Filtered Batch Nr. 1 in 0.01
Did prepro part 1 of 1 in 0.01
Did prepro part 2 of 1 in 0.35
Did prepro part 3 of 1 in 0.00
Merged Users of 1 in 20.75
Extracted TE of 1 in 10.51
Finished Batch Nr. 1 from file part-00000.tsv in 32.27s!
Loaded Batch Nr. 2 in 0.60
Timestamp Filtered Batch Nr. 2 in 0.01
Did prepro part 1 of 2 in 0.01
Did prepro part 2 of 2 in 0.36
Did prepro part 3 of 2 in 0.00
Merged Users of 2 in 19.24
Extracted TE of 2 in 10.39
Finished Batch Nr. 2 from file part-00000.tsv in 30.75s!


{'Q1_reply_rce': -55.06375529232655,
 'Q1_retweet_rce': 2.022846040964088,
 'Q1_retweet_comment_rce': -1779.8738613419546,
 'Q1_like_rce': -33.556324489285075,
 'Q1_reply_avg_prec': 0.15847665901759944,
 'Q1_retweet_avg_prec': 0.3183992278567487,
 'Q1_retweet_comment_avg_prec': 0.012456490593981896,
 'Q1_like_avg_prec': 0.6371322559900724,
 'Q2_reply_rce': -69.87847181000113,
 'Q2_retweet_rce': 6.4290856674497965,
 'Q2_retweet_comment_rce': -1713.128849104653,
 'Q2_like_rce': -26.586873867452155,
 'Q2_reply_avg_prec': 0.15787283026510363,
 'Q2_retweet_avg_prec': 0.30418172967899715,
 'Q2_retweet_comment_avg_prec': 0.0320185234437078,
 'Q2_like_avg_prec': 0.6575847322885718,
 'Q3_reply_rce': -68.04658336242227,
 'Q3_retweet_rce': 11.167206860620848,
 'Q3_retweet_comment_rce': -1096.8625199070543,
 'Q3_like_rce': -15.533225203282308,
 'Q3_reply_avg_prec': 0.17978344581739844,
 'Q3_retweet_avg_prec': 0.3482818652909233,
 'Q3_retweet_comment_avg_prec': 0.015857833740217535,
 'Q3_like_avg_p

In [2]:
from utils.constants import user_group_weights
import torch

In [7]:
groups = [1,2,4,5]

In [8]:
[user_group_weights[it] for it in groups]

[0.9870774328685435,
 0.9705157535873209,
 0.8466576397178028,
 0.4343195950966685]

# Other

In [1]:
import utils.download

In [2]:
utils.download.download_data("validation_urls.txt","data/validation_data", delete_compressed = False)

Uncompressing data/validation_data/part-00000.lzo... done
Uncompressing data/validation_data/part-00001.lzo... done


In [10]:
import pandas as pd
from utils.constants import all_columns, dtypes_of_features, all_features

In [11]:
pd.read_csv("data/validation_data/part-00000.lzo",
                sep='\x01',
                header=None,
                names=all_columns,
                dtype={k: v for k, v in dtypes_of_features.items() if k in all_features},
                nrows = 100000
                )

,bert_base_multilingual_cased_tokens,hashtags,tweet_id,medias,links,domains,type,language,timestamp,a_user_id,...,b_user_id,b_follower_count,b_following_count,b_is_verified,b_account_creation,a_follows_b,reply,retweet,retweet_comment,like
0,101\t157\t39554\t117\t51747\t10479\t27874\t101...,NaN,0C8E7372269942BB173EED7C0A72DA09,NaN,NaN,NaN,Quote,488B32D24BD4BB44172EB981C1BCA6FA,1614523782,8B7BB615A39DF112B0037E960C27F220,...,3044AFDB7E977FB7F62D49C5C97794B7,1128,4028,False,1419763073,False,NaN,NaN,NaN,1.614526e+09
1,101\t14120\t131\t120\t120\t188\t119\t11170\t12...,NaN,858720854891DFED04A5B91758049833,Photo,NaN,NaN,TopLevel,313ECD3A1E5BB07406E4249475C2D6D6,1614335436,74B09D5BC3FCE5CC4DEE34BF049A0EE5,...,23ACD97ACEF57BF6416670BE1133A664,72,115,False,1600058407,False,NaN,NaN,NaN,1.614379e+09
2,101\t56898\t137\t10279\t10908\t10138\t168\t398...,90C52DDF506D1C98EE678E84C08C36AB\tCD8639BABE54...,1988AA4069C46F57990B6983FBD427A6,NaN,D249E94F6D37A7D5E37185E02C94CDFF,9EFF000CDB18B710CDDB43EE1D8C300B,Retweet,E7F038DE3EAD397AEC9193686C911677,1614618153,316C2BE47E445DA930E56C12F3AF31C5,...,99F829F88A12BF8B92EDF11A10B6533C,130,638,False,1257076693,False,NaN,NaN,NaN,NaN
3,101\t56898\t137\t12275\t10738\t11534\t15417\t9...,D704B4128E35F9BC995E701523676542\tCFD3AF9039C6...,89C86150124016236B7D4A286B041210,NaN,NaN,NaN,Retweet,E7F038DE3EAD397AEC9193686C911677,1614714932,C19FC40FBF0AA0BAC4797BD47483349B,...,A3A7B6D928FAD730BA9967B53EA7CC28,234,640,False,1599725409,False,NaN,NaN,NaN,1.614715e+09
4,101\t109821\t24093\t14703\t30118\t11259\t160\t...,NaN,971197A8E595128D8BAF8F3F9D20CFF3,Photo\tPhoto,NaN,NaN,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,1614559113,46846BF25E56F8BA11C0AE36ED31B28F,...,BA94D75FC03B42F88CAE61485E075960,8,208,False,1532879982,False,NaN,NaN,NaN,1.614618e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,101\t49056\t27354\t34935\t69618\t39752\t10230\...,E27EDC9E620D43F17663DCAEAA958585,B434BB652FBBA4D4CA2C8B75CD65D60D,Photo\tPhoto\tPhoto\tPhoto,NaN,NaN,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,1614747844,967E7C2A6A7F277E37C8E77D224B7E18,...,F9414BF0E778E51C5BC48655D0C52583,4542,1401,False,1344160517,True,NaN,NaN,NaN,1.614755e+09
99996,101\t10911\t146\t100\t172\t16863\t29597\t86607...,NaN,710CA52FDB579F8C375D21CF967DD32A,NaN,NaN,NaN,TopLevel,488B32D24BD4BB44172EB981C1BCA6FA,1614670776,04F8A6F207D69EF1E63F4926D1D4DBAB,...,D8462FB3BBD84B0431CF79EC2D106983,738,677,False,1288417538,False,NaN,NaN,NaN,NaN
99997,101\t56898\t137\t14424\t168\t48742\t131\t108\t...,4A99F523FECC5224DFB8B55027794D21\t18B9EA3B7FDC...,ACF411715253EA6B18794E067D62D530,NaN,NaN,NaN,Retweet,E7F038DE3EAD397AEC9193686C911677,1614768600,3AD09AAD3F97E386B12E14772CAA5E67,...,7A7A62AC9D03EC812B0971FFCE113A9B,324,1736,False,1339671557,True,NaN,NaN,NaN,NaN
99998,101\t1894\t88218\t7860\t2316\t1895\t108\t2087\...,35825087788754DC26ED3A6364FD3A55\tD1E4BFB9BD83...,0F6E4AF7D5A70C4C9519D224604EB61C,NaN,A33C298D80264AFAB5612248F785024C,1CE85CC733BA6144D83EDF75F95DCA4E,TopLevel,E7F038DE3EAD397AEC9193686C911677,1614238664,2D2E9B9AD37B72325FDEA86D1197975D,...,3EA329F4500E2FB2DFEF0C482E43C2D5,8,57,False,1548231263,False,NaN,NaN,NaN,NaN


In [3]:
res = next(iter(dl))

Loaded Batch Nr. 1 in 8.59
Timestamp Filtered Batch Nr. 1 in 0.00
Did prepro part 1 of 1 in 0.36
Did prepro part 2 of 1 in 20.40
Did prepro part 3 of 1 in 0.09
Merged Users of 1 in 25.17
Extracted TE of 1 in 14.11
Finished Batch Nr. 1 from file part-00000.tsv in 73.28s!


In [5]:
for col in res[0].columns:
    if col.startswith("n_") or True:
        print(f"{col:60}", end="")

a_follower_count                                            a_following_count                                           a_is_verified                                               b_follower_count                                            b_following_count                                           b_is_verified                                               a_follows_b                                                 bert_token_len                                              photo_count                                                 video_count                                                 gif_count                                                   type_encoding                                               language_encoding                                           a_followers                                                 a_following                                                 b_followers                                                 b_following                             

In [ ]:
for key in res:
    print(f"{key:32}: {res[key]}")